# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [23]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.084247933759138, 4.095385543896985, 6.885704088482912, 4.641264188472668, 5.173874072362352, 6.036183605212701, 4.841362794758202, 4.494678373164601, 3.765203625884911, 5.792104546771135]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [24]:
s2 = np.random.normal( 8, 1 , 10 ).tolist()
print(s2)

[10.433794363071637, 8.515672609317235, 8.93075536582118, 6.44849231924967, 7.5779582767570925, 8.88420132846315, 6.722930858748742, 8.996701206054665, 8.025386305256077, 6.961594705954233]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [25]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.792104546771135, 6.961594705954233, 8.025386305256077, 3.765203625884911, 8.996701206054665, 4.494678373164601, 4.841362794758202, 6.036183605212701, 5.173874072362352, 6.722930858748742]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [26]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.961594705954233, 8.025386305256077, 8.996701206054665, 6.722930858748742]
[5.792104546771135, 3.765203625884911, 4.494678373164601, 4.841362794758202, 6.036183605212701, 5.173874072362352]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 7.67665326900343,  mean_2 = 5.017234503025651
var_1 = 0.8212591177748012,  var_2 = 0.5891323160269984


/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [27]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(  pdf_i[0] * prob_s[1] / prob_x  )
    prob_s1_x.append(  pdf_i[1] * prob_s[1] / prob_x  )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.792104546771135
probability of observing that point if it came from cluster 0 =  0.03491421840766667
probability of observing that point if it came from cluster 1 =  0.2851311382809552
point =  6.961594705954233
probability of observing that point if it came from cluster 0 =  0.33251553001511813
probability of observing that point if it came from cluster 1 =  0.0029202490009122483
point =  8.025386305256077
probability of observing that point if it came from cluster 0 =  0.44389014712122793
probability of observing that point if it came from cluster 1 =  1.476527106047361e-06
point =  3.765203625884911
probability of observing that point if it came from cluster 0 =  5.763861937561827e-06
probability of observing that point if it came from cluster 1 =  0.0707858332786002
point =  8.996701206054665
probability of observing that point if it came from cluster 0 =  0.13348005004703406
probability of observing that point if it came from cluster 1 =  8.372808311046249e-11
point =  

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [28]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([prob_s0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_s0_x), sum([prob_s1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 7.567285295539811,  mean_2 = 4.988965017039782
var_1 = 0.9663364997338928,  var_2 = 0.6019920836350043


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [29]:
prob_s0_x = []
prob_s1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

5.792104546771135
Probability of coming from S_1 = 0.1576130263010919
Probability of coming from S_2 = 0.842386973698908

6.961594705954233
Probability of coming from S_1 = 0.986529125021166
Probability of coming from S_2 = 0.013470874978833995

8.025386305256077
Probability of coming from S_1 = 0.9999919485029327
Probability of coming from S_2 = 8.051497067395792e-06

3.765203625884911
Probability of coming from S_1 = 0.0014241463817017102
Probability of coming from S_2 = 0.9985758536182984

8.996701206054665
Probability of coming from S_1 = 0.9999999982922658
Probability of coming from S_2 = 1.7077342045253492e-09

4.494678373164601
Probability of coming from S_1 = 0.003696323170177214
Probability of coming from S_2 = 0.9963036768298227

4.841362794758202
Probability of coming from S_1 = 0.007943979236241284
Probability of coming from S_2 = 0.9920560207637587

6.036183605212701
Probability of coming from S_1 = 0.3495829653691931
Probability of coming from S_2 = 0.650417034630807

5.1

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [30]:
labels = [0 if prob_s0_x[i] > prob_s1_x[i] else 1 for i in range(len(data))]
print(labels)

[1, 0, 0, 1, 0, 1, 1, 1, 1, 0]
